For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI